In [1]:
%%capture
!pip install --upgrade optuna_integration

In [2]:
import gc
gc.enable()

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

import optuna.integration.lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from sklearn.base import clone
from lightgbm import LGBMRegressor

SEED = 2024

In [3]:
DATA_DIR = '/kaggle/input/playground-series-s4e5'

train = pd.read_csv(f'{DATA_DIR}/train.csv')
test = pd.read_csv(f'{DATA_DIR}/test.csv')
sample_sub = pd.read_csv(f'{DATA_DIR}/sample_submission.csv')

In [4]:
TARGET = 'FloodProbability'

In [5]:
train = train.drop('id', axis=1)
test = test.drop('id', axis=1)

In [6]:
features = list(test.columns)
cat_features = []

In [7]:
# LightGBM dataset
dtrain = lgb.Dataset(
    data=train[features],
    label=train[TARGET],
    feature_name=features,
    categorical_feature=cat_features)

In [8]:
base_params = {
    'objective': 'cross_entropy',
    'metric': 'cross_entropy',
    'learning_rate': 0.1,
    'boosting_type': 'gbdt',
    'force_row_wise': True,
    'verbosity': -1,
    'n_jobs': -1,
    'deterministic': True,
    'random_state': SEED
}

early_stopping = lgb.early_stopping(
    stopping_rounds=100,
    first_metric_only=True,
    verbose=False,
    min_delta=2e-4)

In [9]:
BUDGET = 60 * 60 * 11
NUM_FOLDS = 10

tuner = lgb.LightGBMTunerCV(
    time_budget=BUDGET,
    optuna_seed=SEED,
    params=base_params,
    train_set=dtrain,
    num_boost_round=10000,
    folds=KFold(n_splits=NUM_FOLDS, shuffle=True, random_state=SEED),
    seed=SEED,
    feature_name=features,
    categorical_feature=cat_features,
    callbacks=[early_stopping])

[I 2024-05-01 01:50:19,348] A new study created in memory with name: no-name-3790e3bd-054c-4c03-9293-d1a9d7c03dcb


In [10]:
%%time
tuner.run()

min_child_samples, val_score: 0.688753: 100%|##########| 5/5 [38:12<00:00, 458.56s/it]

CPU times: user 20h 9min 12s, sys: 1h 30min 27s, total: 21h 39min 40s
Wall time: 7h 21min 13s


In [11]:
def comp_metric(y_true, y_pred):
    return r2_score(y_true, y_pred)

def custom_cv(estimator, seed=SEED, verbose=True):
    X_test = test[features]
    
    oof_preds, test_preds = {}, {}
    scores = []

    cv = KFold(n_splits=NUM_FOLDS, shuffle=True, random_state=seed)
    for fold, (train_ids, val_ids) in enumerate(cv.split(train)):
        X_train, y_train = train[features].iloc[train_ids], train[TARGET].iloc[train_ids]
        X_val, y_val = train[features].iloc[val_ids], train[TARGET].iloc[val_ids]
        
        model = clone(estimator)
        model.fit(
            X_train, y_train,
            eval_set=[(X_val, y_val)],
            callbacks=[early_stopping])

        val_preds = model.predict(X_val)
        oof_preds.update(dict(zip(val_ids, val_preds)))
        test_preds[f'fold{fold}'] = model.predict(X_test)

        score = comp_metric(y_val, val_preds)
        scores.append(score)
        if verbose:
            print(f'Fold #{fold:>2}: {score:.5f} ({model.best_iteration_:>4} rounds)')
        _ = gc.collect()

    test_preds = pd.DataFrame.from_dict(test_preds)
    test_preds['mean'] = test_preds.mean(axis=1) # mean of fold-wise predictions
    
    oof_preds = pd.Series(oof_preds).sort_index()
    print(f'\nAvg score: {np.mean(scores):.5f} +/- {np.std(scores):.5f}')
    print(f'OOF score: {comp_metric(train[TARGET], oof_preds):.5f}\n')
    
    return oof_preds, test_preds

In [12]:
%%time
model = LGBMRegressor(**tuner.best_params, n_estimators=10000)
op, tp = custom_cv(model)

Fold # 0: 0.83183 ( 160 rounds)
Fold # 1: 0.83132 ( 163 rounds)
Fold # 2: 0.83267 ( 166 rounds)
Fold # 3: 0.82722 ( 121 rounds)
Fold # 4: 0.83285 ( 163 rounds)
Fold # 5: 0.83144 ( 166 rounds)
Fold # 6: 0.83217 ( 164 rounds)
Fold # 7: 0.83138 ( 164 rounds)
Fold # 8: 0.83267 ( 168 rounds)
Fold # 9: 0.83174 ( 171 rounds)

Avg score: 0.83153 +/- 0.00153
OOF score: 0.83153

CPU times: user 17min 47s, sys: 1.57 s, total: 17min 48s
Wall time: 17min 48s


In [13]:
def create_submission_files(preds, notebook='01'):
    for col in preds.columns:
        sub = sample_sub.copy()
        sub[TARGET] = preds[col]
        sub.to_csv(f'nb{notebook}_{col}.csv', index=False)

In [14]:
create_submission_files(tp)

In [15]:
!head nb01_mean.csv

id,FloodProbability
1117957,0.5717070140324105
1117958,0.45579592025609905
1117959,0.45906267397303846
1117960,0.46471542294081275
1117961,0.4619216507686626
1117962,0.5071689486221109
1117963,0.5399385437014499
1117964,0.5274901201516882
1117965,0.4684161270538416
